IMPORTAÇÕES DE BIBLIOTECAS

In [45]:
import pandas as pd
import scipy.stats as st

LENDO O CSV DAS TABELAS DO BRASILEIRÃO DE 2003 A 2O22

In [46]:
df_br = pd.read_csv('Dados/campeonato-brasileiro-full.csv')

CRIANDO E ORGANIZANDO UM DATAFRAME APENAS COM OS JOGOS DO SÃO PAULO EM 2017 

In [47]:
df_br['data'] = pd.to_datetime(df_br['data'], dayfirst=True)

def determinar_vencedor(win):
    if win['mandante_Placar'] > win['visitante_Placar']:
        return win['mandante']
    elif win['mandante_Placar'] < win['visitante_Placar']:
        return win['visitante']
    else:
        return 'Empate'

def pontuação_sp(row):
    pontos_sao_paulo = 0
    if row['resultado'] == 'Sao Paulo':
        pontos_sao_paulo += 3
    elif row['resultado'] == 'Empate':
        pontos_sao_paulo += 1
    return pontos_sao_paulo

colunas_filtros = ['rodata', 'mandante', 'visitante', 'mandante_Placar', 'visitante_Placar', 'resultado', 'pontos']

CRIANDO UM DATA FRAME COM OS JOGOS PRÉ HERNANES

In [48]:
jogos_antes_hernanes = df_br[(df_br['data'].dt.year == 2017) & (df_br['rodata'] <= 16)].copy()  
pre_hernanes = jogos_antes_hernanes[(jogos_antes_hernanes['mandante'] == 'Sao Paulo') | (jogos_antes_hernanes['visitante'] == 'Sao Paulo')].copy()  
pre_hernanes['resultado'] = pre_hernanes.apply(determinar_vencedor, axis=1)
pre_hernanes['pontos'] = pre_hernanes.apply(pontuação_sp, axis=1)
pre_hernanes = pre_hernanes.loc[:, colunas_filtros].copy()  

pre_hernanes['soma_pontos'] = pre_hernanes['pontos'].cumsum()

pre_hernanes['pontos_possiveis'] = (pre_hernanes['rodata'] ) * 3

print("Jogos Antes de Hernanes:")
print(pre_hernanes.to_string(index=False))

Jogos Antes de Hernanes:
 rodata     mandante   visitante  mandante_Placar  visitante_Placar    resultado  pontos  soma_pontos  pontos_possiveis
      1     Cruzeiro   Sao Paulo                1                 0     Cruzeiro       0            0                 3
      2    Sao Paulo        Avai                2                 0    Sao Paulo       3            3                 6
      3    Sao Paulo   Palmeiras                2                 0    Sao Paulo       3            6                 9
      4  Ponte Preta   Sao Paulo                1                 0  Ponte Preta       0            6                12
      5    Sao Paulo     Vitoria                2                 0    Sao Paulo       3            9                15
      6  Corinthians   Sao Paulo                3                 2  Corinthians       0            9                18
      7        Sport   Sao Paulo                0                 0       Empate       1           10                21
      8    Sao 

CRIANDO UM DATAFRAME COM OS JOGOS PÓS HERNANES

In [49]:
jogos_depois_hernanes = df_br[(df_br['data'].dt.year == 2017) & (df_br['rodata'] >= 17)].copy()  
pos_hernanes = jogos_depois_hernanes[(jogos_depois_hernanes['mandante'] == 'Sao Paulo') | (jogos_depois_hernanes['visitante'] == 'Sao Paulo')].copy()  
pos_hernanes['resultado'] = pos_hernanes.apply(determinar_vencedor, axis=1)
pos_hernanes['pontos'] = pos_hernanes.apply(pontuação_sp, axis=1)
pos_hernanes = pos_hernanes.loc[:, colunas_filtros].copy()  

pos_hernanes['soma_pontos'] = pos_hernanes['pontos'].cumsum()
pos_hernanes['pontos_possiveis'] = (pos_hernanes['rodata'] - 16) * 3

print("\nJogos Depois de Hernanes:")
print(pos_hernanes.to_string(index=False))


Jogos Depois de Hernanes:
 rodata    mandante    visitante  mandante_Placar  visitante_Placar   resultado  pontos  soma_pontos  pontos_possiveis
     17 Botafogo-RJ    Sao Paulo                3                 4   Sao Paulo       3            3                 3
     18   Sao Paulo     Coritiba                1                 2    Coritiba       0            3                 6
     19       Bahia    Sao Paulo                2                 1       Bahia       0            3                 9
     20   Sao Paulo     Cruzeiro                3                 2   Sao Paulo       3            6                12
     21        Avai    Sao Paulo                1                 1      Empate       1            7                15
     22   Palmeiras    Sao Paulo                4                 2   Palmeiras       0            7                18
     23   Sao Paulo  Ponte Preta                2                 2      Empate       1            8                21
     24     Vitoria  

JOGOS EM QUE O SÃO PAULO PONTUOU PELO TOTAL DE JOGOS

In [50]:
n=0
k=0 
p=0

# CALCULANDO O N
for rodadas in pos_hernanes['rodata']:
    n+=1

# CALCULANDO O K
for ponto in pos_hernanes['pontos']:
    if ponto == 1 or ponto == 3:
        k+=1
k-=1

# CALCULANDO O P
jogos_pontuou_pre = 0
jogos_totais_pre = 0

for ponto in pre_hernanes['pontos']:
    if ponto == 1 or ponto == 3:
        jogos_pontuou_pre+=1

for rodadas in pre_hernanes['rodata']:
    jogos_totais_pre+=1

p=jogos_pontuou_pre/jogos_totais_pre

# CALCULANDO A PROBABILIDADE
probabilidade_final1 = 1-st.binom.cdf(n=n, k=k, p=p)
print(probabilidade_final1)
print(f'{probabilidade_final1*100}%')

0.026239395141601562
2.6239395141601562%


PONTO GANHOS DO SÃO PAULO PELO TOTAL DE PONTOS

In [51]:
n=0
k=0
p=0

# CALCULANDO O N
jogos_totais_pos = 0
for rodadas in pos_hernanes['rodata']:
    jogos_totais_pos+=1
n=jogos_totais_pos*3

# CALCULANDO O K
vitorias = 0
empates = 0

for ponto in pos_hernanes['pontos']:
    if ponto == 3:
        vitorias+=1
    elif ponto ==1:
        empates+=1
        
k=(vitorias*3+empates)-1

# CALCULANDO O P
total_pontos_pre=0
vitoria_pre=0
empate_pre=0
pontos_pre=0

for rodadas in pre_hernanes['rodata']:
    total_pontos_pre+=1
total_pontos_pre*=3

for ponto in pre_hernanes['pontos']:
    if ponto == 3:
        vitoria_pre+=1
    elif ponto ==1:
        empate_pre+=1
pontos_pre=(vitoria_pre*3)+empate_pre
p=pontos_pre/total_pontos_pre

# CALCULANDO A PROBABILIDADE
probabilidade_final2 = 1-st.binom.cdf(n=n, k=k, p=p)
print(probabilidade_final2)
print(f'{probabilidade_final2*100}%')

0.001733928952804109
0.1733928952804109%
